In [15]:
import cv2 as cv
import numpy as np

from skimage.color import rgb2gray
import matplotlib.pyplot as plt

import os
import xml.etree.ElementTree as ET

In [2]:
base_dir = 'D:\\Projects\\ship_detection_from_space'
data_dir = base_dir + "/data/MASATI-v2"

In [4]:
def plot_grayscale(img, title, h=8):
    plt.figure(figsize=(h, h))
    _=plt.imshow(img, cmap=plt.get_cmap('gray'))
    _=plt.title(title)

In [5]:
# GO HERE
# https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/


yolo_file_location = f"{base_dir}/yolo_files"

net=cv.dnn.readNetFromDarknet(f'{yolo_file_location}/yolov3.cfg', f'{yolo_file_location}/yolov3.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

ln=net.getLayerNames()

In [16]:
classname_to_id_map = {'ship' : 0,
                       'coast_ship' : 1}

In [17]:
def extract_info_from_xml(file_dir, using_file):

    root = ET.parse(file_dir + using_file).getroot()

    info_dict = {}

    for elem in root:
    # print(elem.tag)
        if elem.tag == 'filename':
            info_dict['filename'] = elem.text

    for subelem in elem:

      #print(subelem.tag)
        if subelem.tag == 'name':
            info_dict['class'] = subelem.text
        if subelem.tag == 'bndbox':
            box_dict = {}
        for i in subelem:
            box_dict[i.tag] = int(i.text)
            info_dict['bndbox'] = box_dict

    return info_dict

In [21]:
def convert_to_yolov5(info_dict):
    print_buffer = []
    try:
        class_id = classname_to_id_map[info_dict["class"]]
    except KeyError:
        print("Invalid Class. Must be one from ", class_name_to_id_map.keys())
        

    # Transform the bbox co-ordinates as per the format required by YOLO v5
    b_center_x = (info_dict['bndbox']["xmin"] + info_dict['bndbox']["xmax"]) / 2 
    b_center_y = (info_dict['bndbox']["ymin"] + info_dict['bndbox']["ymax"]) / 2
    b_width    = (info_dict['bndbox']["xmax"] - info_dict['bndbox']["xmin"])
    b_height   = (info_dict['bndbox']["ymax"] - info_dict['bndbox']["ymin"])
    b_center_x /= 512 
    b_center_y /= 512 
    b_width    /= 512 
    b_height   /= 512 
    #Write the bbox details to the file 
    print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("D:\Projects/ship_detection_from_space/data/yolo_data/labels/", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [22]:
os.getcwd()

'D:\\Projects\\ship_detection_from_space\\code'

In [25]:
using_dir = 'D:\Projects/ship_detection_from_space/data/MASATI-v2/coast_ship_labels/'
all_files = os.listdir(using_dir)
all_dicts = [extract_info_from_xml(using_dir, x) for x in all_files]
n = [convert_to_yolov5(x) for x in all_dicts]